In [1]:
import numpy as np
import xarray as xr
import pystac_client
import planetary_computer
import rasterio as rio
import rioxarray as rxr
from rioxarray.merge import merge_arrays
from urllib.request import urlretrieve
from pyproj import Proj, transform
from os.path import basename, exists, expanduser, join
import geopandas as gpd
from shapely.geometry import shape
import odc.stac

In [2]:
aoi = {
    "type": "Polygon",
    "coordinates":  [
          [
            [-105.8700465489473,
             40.635037173883745],
            [-105.8700465489473,
             39.94338694564476],
            [-105.44917735019361,
              39.94338694564476],
            [-105.44917735019361,
             40.635037173883745],
            [-105.8700465489473,
              40.635037173883745]
          ]
        ]
}

aoi_gpd = gpd.GeoDataFrame({'geometry':[shape(aoi)]}).set_crs(crs="EPSG:4326")

crs = aoi_gpd.estimate_utm_crs()

In [3]:
snowon_date_range = "2024-03-01/2024-04-01"
snowoff_date_range = "2023-09-01/2023-10-01"

In [4]:
stac = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

## grab S1 data

In [6]:
# snow on ds
search = stac.search(
    intersects=aoi,
    datetime=snowon_date_range,
    collections=["sentinel-1-rtc"]
)

items = search.item_collection()

snowon_s1_ds = odc.stac.load(items,chunks={"x": 2048, "y": 2048},
                             crs=crs,
                             resolution=50,
                             bbox=aoi_gpd.total_bounds,
                             groupby='sat:absolute_orbit')

print(f"Returned {len(snowon_s1_ds.time)} acquisitions")

# limit to morning acquisitions
snowon_s1_ds = snowon_s1_ds.where(snowon_s1_ds.time.dt.hour > 11, drop=True)

# compute median
snowon_s1_ds = snowon_s1_ds.median(dim='time').squeeze().compute()

# rename variables
snowon_s1_ds = snowon_s1_ds.rename({
    'vv': 'snowon_vv',
    'vh': 'snowon_vh'
})

# # mask negative areas
# snowon_s1_ds = snowon_s1_ds.where(snowon_s1_ds.vh > 0)
# snowon_s1_ds = snowon_s1_ds.where(snowon_s1_ds.vv > 0)

Returned 10 acquisitions


In [7]:
# snow off ds 
search = stac.search(
    intersects=aoi,
    datetime=snowoff_date_range,
    collections=["sentinel-1-rtc"]
)

items = search.item_collection()

snowoff_s1_ds = odc.stac.load(items,chunks={"x": 2048, "y": 2048},
                              like=snowon_s1_ds,
                              groupby='sat:absolute_orbit')

print(f"Returned {len(snowoff_s1_ds.time)} acquisitions")

# limit to morning acquisitions
snowoff_s1_ds = snowoff_s1_ds.where(snowoff_s1_ds.time.dt.hour > 11, drop=True)

# # mask negative areas
# snowoff_s1_ds = snowoff_s1_ds.where(snowoff_s1_ds.vh > 0)
# snowoff_s1_ds = snowoff_s1_ds.where(snowoff_s1_ds.vv > 0)

# compute median
snowoff_s1_ds = snowoff_s1_ds.median(dim='time').squeeze().compute()

snowoff_s1_ds = snowoff_s1_ds.rename({
    'vv': 'snowoff_vv',
    'vh': 'snowoff_vh'
})

Returned 10 acquisitions


/srv/conda/envs/notebook/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


## grab s2 data

In [8]:
search = stac.search(
    intersects=aoi,
    datetime=snowon_date_range,
    collections=["sentinel-2-l2a"],
    query={"eo:cloud_cover": {"lt": 25}},
)

items = search.item_collection()

s2_ds = odc.stac.load(items,chunks={"x": 2048, "y": 2048},
                      like=snowon_s1_ds,
                      groupby='solar_day').where(lambda x: x > 0, other=np.nan)

print(f"Returned {len(s2_ds.time)} acquisitions")

# compute median
s2_ds = s2_ds.median(dim='time').squeeze().compute()

Returned 3 acquisitions


## grab cop30 data

In [9]:
search = stac.search(
    collections=["cop-dem-glo-30"],
    intersects=aoi
)

items = search.item_collection()
    
data = []
for item in items:
    dem_path = planetary_computer.sign(item.assets['data']).href
    data.append(rxr.open_rasterio(dem_path))
cop30_da = merge_arrays(data)
cop30_ds = cop30_da.rename('elevation').squeeze().to_dataset()

# reproject to match radar dataset
cop30_ds = cop30_ds.rio.reproject_match(snowon_s1_ds, resampling=rio.enums.Resampling.bilinear).compute()

## grab fcf data

In [10]:
def url_download(url, out_fp, overwrite = False):
    # check if file already exists
    if not exists(out_fp) or overwrite == True:
            urlretrieve(url, out_fp)
    # if already exists. skip download.
    else:
        print('file already exists, skipping')

def download_fcf(out_fp):
    # this is the url from Lievens et al. 2021 paper
    fcf_url = 'https://zenodo.org/record/3939050/files/PROBAV_LC100_global_v3.0.1_2019-nrt_Tree-CoverFraction-layer_EPSG-4326.tif'
    # download just forest cover fraction to out file
    url_download(fcf_url, out_fp)

fcf_path ='/tmp/fcf_global.tif'
download_fcf(fcf_path)

# open as dataArray and return
fcf_ds = rxr.open_rasterio(fcf_path)

# clip to aoi
fcf_ds = fcf_ds.rio.clip_box(*aoi_gpd.total_bounds,crs="EPSG:4326") 

# promote to dataset
fcf_ds = fcf_ds.rename('fcf').squeeze().to_dataset()

# reproject to match radar dataset
fcf_ds = fcf_ds.rio.reproject_match(snowon_s1_ds, resampling=rio.enums.Resampling.bilinear)

# set values above 100 to nodata
fcf_ds['fcf'] = fcf_ds['fcf'].where(fcf_ds['fcf'] <= 100, np.nan)/100

file already exists, skipping


## combine datasets

In [11]:
ds_list = [snowon_s1_ds, snowoff_s1_ds, s2_ds, cop30_ds, fcf_ds]
ds = xr.merge(ds_list, compat='override', join='override').squeeze()

## calculate additional data variables

In [12]:
# calculate cross ratios
def db_scale(x, epsilon=1e-10):
    # Add epsilon only where x is zero
    x_with_epsilon = np.where(x==0, epsilon, x)
    # Calculate the logarithm
    log_x = 10 * np.log10(x_with_epsilon)
    # Set the areas where x was originally zero back to zero
    log_x[x==0] = 0
    return log_x

In [13]:
# radar data variables
# convert to decibels
ds['snowon_vv'] = (('y', 'x'), db_scale(ds['snowon_vv']))
ds['snowon_vh'] =  (('y', 'x'),db_scale(ds['snowon_vh']))
ds['snowoff_vv'] =  (('y', 'x'),db_scale(ds['snowoff_vv']))
ds['snowoff_vh'] =  (('y', 'x'),db_scale(ds['snowoff_vh']))

# calculate variables
ds['snowon_cr'] = ds['snowon_vh'] - ds['snowon_vv']
ds['snowoff_cr'] = ds['snowoff_vh'] - ds['snowoff_vv']
ds['delta_cr'] = ds['snowon_cr'] - ds['snowoff_cr']

In [14]:
# s2 band indices
ds['ndvi'] = (ds['B08'] - ds['B04'])/(ds['B08'] + ds['B04'])
ds['ndsi'] = (ds['B03'] - ds['B11'])/(ds['B03'] + ds['B11'])
ds['ndwi'] = (ds['B03'] - ds['B08'])/(ds['B03'] + ds['B08'])

In [15]:
# latitude, longitude
# define projections
utm_proj = Proj(proj='utm', zone=crs.name[-3:-1], ellps='WGS84') ## NOTE hardcoded utm for now, adjust before use
wgs84_proj = Proj(proj='latlong', datum='WGS84')

x, y = np.meshgrid(ds['x'].values, ds['y'].values)
lon, lat = transform(utm_proj, wgs84_proj, x, y)
ds['latitude'] = (('y', 'x'), lat)
ds['longitude'] = (('y', 'x'), lon)

In [16]:
# # dowy
# def calc_dowy(doy):
#     'calculate day of water year from day of year'
#     if doy < 274:
#         dowy = doy + (365-274)
#     elif doy >= 274:
#         dowy = doy-274
#     return dowy

# ## NOTE think about date and fix this
# dowy_1d = calc_dowy(pd.to_datetime(fn.split('_')[4]).dayofyear)
# dowy = torch.full_like(aso_sd, dowy_1d)

## write out to data file

In [18]:
ds.to_netcdf(f'../../data/application_tmp/RMNP.nc')